In [ ]:
# Import necessary libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pandas as pd

# Ensure required directories exist
os.makedirs('models', exist_ok=True)
os.makedirs('plots', exist_ok=True)


In [ ]:
# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target
y_categorical = tf.keras.utils.to_categorical(y, num_classes=3)

# Split the data (keeping original labels for confusion matrix)
X_train, X_test, y_train, y_test, y_train_labels, y_test_labels = train_test_split(
    X, y_categorical, y, test_size=0.2, random_state=42
)
print('Data loaded and split.')


In [ ]:
# Build a simple Sequential model 2 hidden layers [subject to change]
model = Sequential([
    Dense(20, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(10, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print('Model built.')


In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_split=0.1, verbose=1)
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Accuracy: {accuracy}')


In [ ]:
# Generate predictions and compute confusion matrix
y_pred_prob = model.predict(X_test)
y_pred = y_pred_prob.argmax(axis=1)
cm = confusion_matrix(y_test_labels, y_pred)
acc = accuracy_score(y_test_labels, y_pred)
print('Confusion Matrix:')
print(cm)
print(f'Accuracy: {acc}')


In [ ]:
# Plot training accuracy and validation loss
epochs = range(1, len(history.history['accuracy']) + 1)

# Accuracy Plot
plt.figure(figsize=(8,6))
plt.plot(epochs, history.history['accuracy'], label='Training Accuracy')
plt.plot(epochs, history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
accuracy_plot_path = os.path.join('plots', 'accuracy_plot.png')
plt.savefig(accuracy_plot_path)
plt.close()

# Validation Loss Plot
plt.figure(figsize=(8,6))
plt.plot(epochs, history.history['loss'], label='Training Loss')
plt.plot(epochs, history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
val_loss_plot_path = os.path.join('plots', 'val_loss_plot.png')
plt.savefig(val_loss_plot_path)
plt.close()

# Confusion Matrix Heatmap
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
cm_plot_path = os.path.join('plots', 'confusion_matrix.png')
plt.savefig(cm_plot_path)
plt.close()

print('Plots generated and saved in the plots directory.')


In [ ]:
# Identify one correctly classified and one misclassified sample
correct_indices = [i for i, (pred, true) in enumerate(zip(y_pred, y_test_labels)) if pred == true]
incorrect_indices = [i for i, (pred, true) in enumerate(zip(y_pred, y_test_labels)) if pred != true]

# Plot for correctly classified sample
if correct_indices:
    correct_idx = correct_indices[0]
    sample_prob = y_pred_prob[correct_idx]
    sample_true = y_test_labels[correct_idx]
    sample_pred = y_pred[correct_idx]
    plt.figure(figsize=(6,4))
    plt.bar(range(3), sample_prob)
    plt.title(f'Correctly Classified Sample: True Label = {sample_true}, Predicted = {sample_pred}')
    plt.xlabel('Class')
    plt.ylabel('Probability')
    correct_sample_path = os.path.join('plots', 'correct_sample.png')
    plt.savefig(correct_sample_path)
    plt.close()
else:
    print("No correctly classified sample found.")

# Plot for misclassified sample
if incorrect_indices:
    incorrect_idx = incorrect_indices[0]
    sample_prob = y_pred_prob[incorrect_idx]
    sample_true = y_test_labels[incorrect_idx]
    sample_pred = y_pred[incorrect_idx]
    plt.figure(figsize=(6,4))
    plt.bar(range(3), sample_prob)
    plt.title(f'Misclassified Sample: True Label = {sample_true}, Predicted = {sample_pred}')
    plt.xlabel('Class')
    plt.ylabel('Probability')
    incorrect_sample_path = os.path.join('plots', 'incorrect_sample.png')
    plt.savefig(incorrect_sample_path)
    plt.close()
else:
    print("No misclassified sample found.")


In [ ]:
# Save the trained model
model.save('models/model.h5')
print('Model saved to models/model.h5')


In [ ]:
# Generate a markdown report with key metrics, links to plots, and sample prediction images
report_content = f"""
# Model Training Report

## Test Accuracy

Test Accuracy: {accuracy:.4f}

## Confusion Matrix

{cm}


## Plots

- [Accuracy and Validation Loss Plot](plots/accuracy_plot.png)
- [Validation Loss Plot](plots/val_loss_plot.png)
- [Confusion Matrix](plots/confusion_matrix.png)

## Sample Predictions

**Correctly Classified Sample:**  
![Correct Sample](plots/correct_sample.png)

**Misclassified Sample:**  
![Incorrect Sample](plots/incorrect_sample.png)

## Training History

{pd.DataFrame(history.history).to_markdown()}

"""

with open('report.md', 'w') as f:
    f.write(report_content)

print('Report generated as report.md')